## コンテキストマネージャー

with文に渡すオブジェクトをコンテキストマネージャーと呼ぶ。
これは、メソッド<m-b>\_\_enter\_\_()</m-b>、<m-b>\_\_exit\_\_()</m-b>の特殊メソッドを実装したクラスのインスタンスである。

In [3]:
# open()で取得するコンテキストマネージャーの確認
file_obj = open('sample.txt')
file_dir = dir(file_obj)
print('__enter__' in file_dir)
print('__exit__' in file_dir)

True
True


In [15]:
# 独自のコンテキストマネージャーでopen()をラップ
class MyOpenContextManager:
    def __init__(self, file_path):
        self.file_path = file_path
    def __enter__(self):
        print(f'file open: {self.file_path}')
        self.file_obj = open(self.file_path)
        return self.file_obj
    def __exit__(self, type, value, traceback):
        if type:
            print(f'error occurred: type={type}, value={value}')
        print(f'close file: {self.file_path}')
        self.file_obj.close()

try:
    with MyOpenContextManager('sample.txt') as f:
        print(f.read())
except:
    pass
print()
try:
    with MyOpenContextManager('sample.txt') as f:
        print(f.readd())
except:
    pass

file open: sample.txt
This is sample text.
close file: sample.txt

file open: sample.txt
error occurred: type=<class 'AttributeError'>, value='_io.TextIOWrapper' object has no attribute 'readd'
close file: sample.txt


In [29]:
# デコレータを用いて関数をコンテキストマネージャとして扱う
import contextlib
import traceback

@contextlib.contextmanager
def my_open_context_manager(file_path):
    file_obj = open(file_path, 'r')
    try:
        print('file open: {file_path}')
        yield file_obj
    except Exception as ex:
        traceback.print_exception(ex)
    finally:
        file_obj.close()
        print(f'close file: {file_path}')

try:
    with my_open_context_manager('sample.txt') as f:
        print(f.read())
except:
    pass
print()
try:
    with my_open_context_manager('sample.txt') as f:
        print(f.readd())
except:
    pass

file open: {file_path}
This is sample text.
close file: sample.txt

file open: {file_path}
close file: sample.txt


Traceback (most recent call last):
  File "C:\Users\kgrmr\AppData\Local\Temp\ipykernel_16580\2761131906.py", line 10, in my_open_context_manager
    yield file_obj
  File "C:\Users\kgrmr\AppData\Local\Temp\ipykernel_16580\2761131906.py", line 25, in <module>
    print(f.readd())
          ^^^^^^^
AttributeError: '_io.TextIOWrapper' object has no attribute 'readd'


In [33]:
# コンテキストマネージャを用いて、実行時間を測定するサンプル
import contextlib
import time

@contextlib.contextmanager
def timed():
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print(f'process duration: {end - start}s')

with timed():
    time.sleep(0.5)

process duration: 0.5007326602935791s


In [42]:
# contextlibモジュールのその他の機能
import contextlib
import os

# 指定の例外を無視する
with contextlib.suppress(FileNotFoundError):
    os.remove('存在しないファイル.txt')

# 標準出力/標準エラーの出力先を変える
with open('stdout.log', 'w') as f:
    with contextlib.redirect_stdout(f):
        print('hogehoge')
with open('stdout.log', mode='r') as f:
    print(f'file content: {f.read()}')

file content: hogehoge

